In [ ]:
!git clone -l -s git://github.com/srgynmv/EmoteNN.git emotenn
%cd emotenn

In [ ]:
%tensorflow_version 2.x
import os
import numpy as np
from emotenn import constants as ct, load_utils as lu, train_utils as tu
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import callbacks
import tensorflow as tf

In [ ]:
X, Y = lu.load_dataset(ct.FER2013)
img_count, w, h, _ = X.shape

X_train, X_valid, X_test, Y_train, Y_valid, Y_test = tu.split_dataset(X, Y, test_size=0.15, valid_size=0.2, align_by=512)
print(f'Train data dimensions: {X_train.shape}')
print(f'Validation data dimensions: {X_valid.shape}')
print(f'Test data dimensions: {X_test.shape}')

In [ ]:
strategy = tu.get_distribution_strategy()

In [ ]:
from models import model as builder

features_count = 64
labels_count = 7

with strategy.scope():
    model = builder.build_model(features_count, labels_count, w, h)
    model.compile(loss=categorical_crossentropy,
                  optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
                  metrics=['accuracy'])

In [ ]:
batch_size = 512
epochs = 10
model_name = 'model'

callbacks = tu.get_callbacks(model_name)
train_history = model.fit(X_train,
                          Y_train,
                          batch_size=batch_size,
                          epochs=epochs,
                          validation_data=(X_valid, Y_valid),
                          shuffle=True)

In [ ]:
score = model.evaluate(X_test, Y_test)
print("Dataset {}: {:.2f}%".format(model.metrics_names[1], score[1] * 100))

In [ ]:
tu.save_results(model, train_history, model_name)